In [1]:
import numpy as np
#standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer

In [1]:
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
# molecule = "H 0.0 0.0 -0.3625; H 0.0 0.0 0.3625 "
driver = PySCFDriver(atom=molecule)

NameError: name 'PySCFDriver' is not defined

In [21]:
qmolecule = driver.run()

In [40]:
fct = [FreezeCoreTransformer(freeze_core=False)]#(freeze_core=True, remove_orbitals=[3,4])]

In [41]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
problem = ElectronicStructureProblem(driver, fct)
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)

# Mapping Fermions to Qubits
#num_particles = (problem.molecule_data_transformed.num_alpha,
             #problem.molecule_data_transformed.num_beta)
num_particles = problem.num_particles
qubit_op = converter.convert(main_op, num_particles = problem.num_particles)
from qiskit_nature.circuit.library import HartreeFock


num_spin_orbitals = problem.num_spin_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

In [42]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Parameters for q-UCC antatze
num_particles = problem.num_particles
num_spin_orbitals = problem.num_spin_orbitals
n = qubit_op.num_qubits
qc = QuantumCircuit(qubit_op.num_qubits)
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
#the variational parameter
p=1
for i in range(n):
    theta = Parameter(f"ry_theta{p}" )
    qc.ry(theta, i)
    p += 1
qubit_label = 0
# qc.ry(theta, range(n))
#qc.rz(theta, range(n))
for i in range(n-1):
    qc.cz(i, i+1)
for i in range(n):
    theta = Parameter(f"ry_theta{p}" )
    qc.ry(theta, i)
    p += 1
#qc.rz(theta, range(n))

# Add the initial state
ansatz = qc
ansatz.compose(init_state, front=True, inplace=True)

In [43]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

Exact electronic energy -1.867120978341268
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.867120978341
  - computed part:      -1.867120978341
  - FreezeCoreTransformer extracted energy part: 0.0
~ Nuclear repulsion energy (Hartree): 0.729899601269
> Total ground state energy (Hartree): -1.137221377072
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


In [44]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
optimizer = COBYLA(maxiter=15000)
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
def callback(eval_count, parameters, mean, std):  
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 124,
    'eigenstate': array([ 5.50373743e-05+2.57245038e-17j,  9.93981903e-01-5.70512798e-18j,
       -1.09544394e-01+1.27862942e-17j, -4.60683152e-06-2.83571824e-18j]),
    'eigenvalue': (-1.867120975956159+0j),
    'optimal_circuit': None,
    'optimal_parameters': {   Parameter(ry_theta4): 0.4368415392393307,
                              Parameter(ry_theta2): 0.4857237209833987,
                              Parameter(ry_theta1): 0.48529449177375367,
                              Parameter(ry_theta3): -0.43649110091540416},
    'optimal_point': array([ 0.48529449,  0.48572372, -0.4364911 ,  0.43684154]),
    'optimal_value': -1.867120975956159,
    'optimizer_evals': None,
    'optimizer_result': None,
    'optimizer_time': 0.580376148223877}


In [25]:
ansatz.draw()

┌───┐      ┌───────────────┐   ┌───────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(ry_theta1) ├─■─┤ Ry(ry_theta5) ├─────────────────»
           ├───┤      ├───────────────┤ │ └───────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(ry_theta2) ├─■─────────■────────┤ Ry(ry_theta6) ├»
     ┌─────┴───┴─────┐└───────────────┘           │        └───────────────┘»
q_2: ┤ Ry(ry_theta3) ├────────────────────────────■────────────────■────────»
     ├───────────────┤                                             │        »
q_3: ┤ Ry(ry_theta4) ├─────────────────────────────────────────────■────────»
     └───────────────┘                                                      »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(ry_theta7) ├
«     ├───────────────┤
«q_3: ┤ Ry(ry_theta8) ├
«     └───────────────┘

In [26]:
parameter = result.optimal_parameters

In [27]:
op_list = qubit_op.primitive.group_commuting()

In [28]:
def MeasureCircuit(Sum_Op):
    # Determine how many commute groups are in the SummedOp
    num_terms = len(Sum_Op)

    # Find the Paulis with least number of I in it(shoud be 0).
    # The problem is here. In this case, not not all the subgroups have a term with no I in it. So we have to loop over all
    # of the terms to construct a Pauli string.
    Pauli = ''

    for i in range(Sum_Op.num_qubits):
        intermed = []
        for j in range(num_terms):
            intermed.append(Sum_Op[j].to_label()[i])
        print(intermed)
        if 'X' in intermed:
            Pauli += 'X'
        elif 'Y' in intermed:
            Pauli += 'Y'
        else:
            Pauli += 'Z'

    if len(Pauli) != Sum_Op.num_qubits:
        raise Exception('The length does not equal, traverse has problem.')

    Pauli_string = Pauli[::-1]  # This has reversed the order.
    # Now Pauli_string is the target that we should use to construct the measurement circuit.

    qc = QuantumCircuit(Sum_Op.num_qubits, Sum_Op.num_qubits)
    qc.barrier()
    print(Pauli_string)

    for i in range(Sum_Op.num_qubits):
        if Pauli_string[i] == 'X':
            qc.u(np.pi / 2, 0, np.pi, i)
        if Pauli_string[i] == 'Y':
            qc.u(np.pi / 2, 0, np.pi / 2, i)
        else:
            None

    qc.measure(range(Sum_Op.num_qubits), range(Sum_Op.num_qubits));

    return qc


In [29]:
LiH_list = []
for commute_set in op_list:
    temp_list = []
    for i in range(0, len(commute_set.paulis)):
        temp_list.append((commute_set.paulis[i].to_label(),commute_set.coeffs[i]))
    LiH_list.append(temp_list)

In [30]:
LiH_list

[[('ZXZX', (-0.003139482375506528+0j)),
  ('IXZX', (-0.003139482375506528+0j)),
  ('ZXIX', (0.003139482375506528+0j)),
  ('IXIX', (0.003139482375506528+0j)),
  ('ZIZX', (-0.010681856282947617+0j)),
  ('ZIIX', (0.010681856282947617+0j)),
  ('ZXZI', (-0.010681856282947617+0j)),
  ('IXZI', (-0.010681856282947617+0j)),
  ('ZIZI', (-0.11384335176464518+0j)),
  ('IIZX', (-0.002941141087330846+0j)),
  ('IIIX', (0.002941141087330846+0j)),
  ('ZXII', (0.0029411410873308465+0j)),
  ('IXII', (0.0029411410873308465+0j)),
  ('ZIII', (-0.365258690216037+0j)),
  ('IIZI', (0.3652586902160371+0j)),
  ('IIII', (-0.20316606150559116+0j))],
 [('XXZX', (0.008499158469817442+0j)),
  ('YYZX', (-0.008499158469817442+0j)),
  ('XXIX', (-0.008499158469817442+0j)),
  ('YYIX', (0.008499158469817442+0j)),
  ('ZZZX', (0.001697464962393525+0j)),
  ('ZZIX', (-0.001697464962393525+0j)),
  ('XXZI', (0.034389748140473936+0j)),
  ('YYZI', (-0.034389748140473936+0j)),
  ('ZZZI', (-0.06044012857315994+0j)),
  ('XXII', (-0.0

In [31]:
op_list[-1].paulis

PauliList(['XZXZ', 'XIXZ', 'XZXI', 'XIXI', 'IZXZ', 'IZXI', 'XZIZ',
           'XIIZ', 'IZIZ'])

In [32]:
op_list[-1].coeffs

array([-0.00658758+0.j,  0.00658758+0.j, -0.00658758+0.j,  0.00658758+0.j,
       -0.01277933+0.j, -0.01277933+0.j, -0.01277933+0.j,  0.01277933+0.j,
       -0.12274244+0.j])

based on the paulis we can generate measurment circuit to change the basis

In [33]:
# This is a script I wrote to better manipulate the measurement process
from Measure import MeasureCircuit
meas_qc = MeasureCircuit(op_list[-1].paulis)

['X', 'X', 'X', 'X', 'I', 'I', 'X', 'X', 'I']
['Z', 'I', 'Z', 'I', 'Z', 'Z', 'Z', 'I', 'Z']
['X', 'X', 'X', 'X', 'X', 'X', 'I', 'I', 'I']
['Z', 'Z', 'I', 'I', 'Z', 'I', 'Z', 'Z', 'Z']


In [34]:
meas_qc.draw()

░               ┌─┐         
q_0: ─░───────────────┤M├─────────
      ░ ┌────────────┐└╥┘   ┌─┐   
q_1: ─░─┤ U(π/2,0,π) ├─╫────┤M├───
      ░ └────────────┘ ║ ┌─┐└╥┘   
q_2: ─░────────────────╫─┤M├─╫────
      ░ ┌────────────┐ ║ └╥┘ ║ ┌─┐
q_3: ─░─┤ U(π/2,0,π) ├─╫──╫──╫─┤M├
      ░ └────────────┘ ║  ║  ║ └╥┘
c_0: ══════════════════╩══╬══╬══╬═
                          ║  ║  ║ 
c_1: ═════════════════════╬══╩══╬═
                          ║     ║ 
c_2: ═════════════════════╩═════╬═
                                ║ 
c_3: ═══════════════════════════╩═

In [35]:
op_list[0].paulis[0].to_label()

'ZXZX'